(https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/Cassandra1.jpeg/800px-Cassandra1.jpeg)
## Notebook to run batch Kaissandra methods

With this notebook you can run train, test or alternated train and test methods


In [1]:
import sys
import os

this_path = os.getcwd()
path = '\\'.join(this_path.split('\\')[:-1])+'\\'
if path not in sys.path:
    sys.path.insert(0, path)
    print(path+" added to python path")
else:
    print(path+" already in python path")
    
%load_ext autoreload
%autoreload 2

C:\Users\mgutierrez\root\Projects\SDC\py\kaissandra\ added to python path


In [2]:
# Import modules
import time

from kaissandra.config import configuration, retrieve_config
from kaissandra.testRNN import test_RNN
from kaissandra.trainRNN import train_RNN
from kaissandra.inputs import Data, run_train_test, build_and_test, build_and_train


#%reload_ext autoreload

C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Get config structure

In [4]:
params = {'config_name':'C0401',
          
          'movingWindow':100,
          'nEventsPerStat':1000,
          'lB':1300,
          'feature_keys_manual':[],
          'feature_keys_tsfresh':[i for i in range(37,48)]+[49,50]+[i for i in range(52,68)],
           'var_feat_keys':[i for i in range(68,93)],
          
          'if_build_IO':True,
          'from_stats_file':False,
          'IDweights':'000401',
          'inverse_load':True,
          
          'IDresults':'100401',
          'startFrom':-1,
          'endAt':-1,
          'save_journal':False}

config = configuration(params)

WARNING! Arguments not taken into consideration
Config file ../../config/C0401.config exists. Loaded from disk


## Build inputs/outputs and train Network

In [ ]:
build_and_train(config)

## Build inputs/outputs and test Network

In [ ]:
build_and_test(config)

## Automately train and test in alternative manner

In [ ]:
# Automate train/test from variation features
if_train = True
if_test = True
its = 20
configs = [config['config_name']]

configs_list = []
# load configuration files
for config_name in configs:
    configs_list.append(retrieve_config(config_name))
# run train/test
for config in configs_list:
    # set automation-specific config fields
    config['num_epochs'] = 1
    config['startFrom'] = -1
    config['endAt'] = -1
    run_train_test(config, its, if_train, if_test)

Config file ../../config/C0401.config loaded from disk
Iteration 0 of 19
IDweights: 000401
IDresults: 100401
1 AUDCAD
	s 0 of 21. From 2016.01.04 00:00:08 to 2016.10.13 15:41:59
	s 1 of 21. From 2016.10.13 17:52:00 to 2016.12.23 23:59:59
	s 2 of 21. From 2016.12.26 06:00:00 to 2016.12.30 23:58:30
	s 3 of 21. From 2017.01.02 09:00:00 to 2017.01.26 23:59:59
	s 4 of 21. From 2017.01.30 00:05:00 to 2017.09.05 23:59:59
	s 5 of 21. From 2017.09.06 20:25:01 to 2017.11.10 09:38:15
	s 6 of 21. From 2017.11.10 10:05:52 to 2017.11.24 23:58:59
	s 7 of 21. From 2017.11.27 00:05:00 to 2017.12.08 23:58:59
	s 8 of 21. From 2017.12.11 00:05:00 to 2017.12.12 04:13:07
	s 9 of 21. From 2017.12.12 09:55:53 to 2017.12.22 23:58:59
	s 10 of 21. From 2017.12.26 03:00:00 to 2017.12.29 23:58:42
	s 11 of 21. From 2018.01.02 03:00:00 to 2018.01.10 21:25:25
	s 12 of 21. From 2018.01.11 14:39:34 to 2018.01.25 08:17:37
	s 13 of 21. From 2018.01.25 08:53:48 to 2018.03.08 23:58:59
	s 14 of 21. From 2018.03.09 00:05:00 

100%|████████████████████████████████████████████████████████████████████████████| 32768/32768 [06:41<00:00, 81.62it/s]


Time training Chunck 0 of 14: 438 s


100%|████████████████████████████████████████████████████████████████████████████| 32768/32768 [07:01<00:00, 77.76it/s]


Time training Chunck 1 of 14: 460 s


...

100%|████████████████████████████████████████████████████████████████████████████| 32768/32768 [10:06<00:00, 54.06it/s]


Time training Chunck 9 of 14: 642 s


100%|████████████████████████████████████████████████████████████████████████████| 32768/32768 [09:15<00:00, 58.98it/s]


Time training Chunck 10 of 14: 592 s


 40%|██████████████████████████████▌                                             | 13188/32768 [03:39<05:38, 57.83it/s]

In [9]:
config['assets']=[1,2,3,4,7,8,10,11,12,13,14,15,16,17,19,27,28,29,30,31,32]

In [7]:
from kaissandra.RNN import modelRNN

data=Data(movingWindow=config['movingWindow'],
              nEventsPerStat=config['nEventsPerStat'],
              lB=config['lB'], 
              dateTest=config['dateTest'],
              assets=config['assets'],
              channels=config['channels'],
              max_var=config['max_var'],
              feature_keys_manual=config['feature_keys_manual'],
              feature_keys_tsfresh=config['feature_keys_tsfresh'],
              var_feat_keys=config['var_feat_keys'])
    
model=modelRNN(data,
                   size_hidden_layer=config['size_hidden_layer'],
                   L=config['L'],
                   size_output_layer=config['size_output_layer'],
                   keep_prob_dropout=config['keep_prob_dropout'],
                   miniBatchSize=config['miniBatchSize'],
                   outputGain=config['outputGain'],
                   commonY=config['commonY'],
                   lR0=config['lR0'],
                   num_epochs=config['num_epochs'])
print(data.feature_keys_manual)
print(len(data.feature_keys_tsfresh)+len(data.var_feat_keys))
print(len(data.var_feat_keys))
print(data.n_feats_tsfresh+len(data.var_feat_keys))
print(data.nFeatures)

[]
54
25
99
99


In [11]:
# test individual assets
assets = [16,17,27,29,30,32]
for asset in assets:
    ass_name = Data().AllAssets[str(asset)]
    try:
        config = configuration({'config_name':'C287INVO_'+ass_name,
                               'assets':[asset],
                               'IDresults':'C287INVO_'+ass_name})
    except OSError:
        config = retrieve_config('C287INVO_'+ass_name)
    test_RNN(config)

Config file ../config/C287INVO_USDCAD.config loaded from disk
IDresults: C287INVO_USDCAD
INFO:tensorflow:Restoring parameters from ../RNN/weights/000287INVO/000007/000007
Parameters loaded. Epoch 7
Epoch 7 of 13. Getting output...
Chunck 1 of 1
Getting results
Epoch = 7. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.5
J_test = 4.5608315, J_train = 4.2515653775989675, Accuracy=0.6803612704391747
RD = 836 NZ = 1561 NZA = 2537 pNZ = 1.336% pNZA = 2.172% AD = 53.56% ADA = 32.95% NO = 329 GSP = 56.84% NSP = 48.33%
SI2 = -32.95 SI = -21.95 eGROI = 1.25% eROI = -2.78% eROI2 = -5.33% eROI3 = -8.62% mSpread = 0.0123%
Epoch = 7. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.6
RD = 298 NZ = 493 NZA = 818 pNZ = 0.422% pNZA = 0.700% AD = 60.45% ADA = 36.43% NO = 205 GSP = 59.02% NSP = 52.20%
SI2 = -12.75 SI = -5.75 eGROI = 3.29% eROI = 0.71% eROI2 = -0.81% eROI3 = -2.86% mSpread = 0.0126%
Epoch = 7. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.7
RD = 107 NZ = 150 NZA = 237 pNZ

C:\Users\mgutierrez\root\Projects\SDC\py\git\results2.py:619: RuntimeWarning: divide by zero encountered in double_scalars
  eROIs = eROIs+GROI-fixed_spread_ratios


Epoch = 7. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.5
RD = 327 NZ = 569 NZA = 837 pNZ = 0.487% pNZA = 0.717% AD = 57.47% ADA = 39.07% NO = 154 GSP = 55.84% NSP = 50.65%
SI2 = -12.70 SI = -6.70 eGROI = 1.58% eROI = -0.17% eROI2 = -1.50% eROI3 = -3.04% mSpread = 0.0113%
Epoch = 7. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.6
RD = 164 NZ = 263 NZA = 391 pNZ = 0.225% pNZA = 0.335% AD = 62.36% ADA = 41.94% NO = 114 GSP = 55.26% NSP = 48.25%
SI2 = -8.70 SI = -7.70 eGROI = 1.28% eROI = -0.07% eROI2 = -1.00% eROI3 = -2.14% mSpread = 0.0119%
Epoch = 7. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.7
RD = 64 NZ = 90 NZA = 126 pNZ = 0.077% pNZA = 0.108% AD = 71.11% ADA = 50.79% NO = 61 GSP = 70.49% NSP = 62.30%
SI2 = 2.45 SI = 4.45 eGROI = 0.87% eROI = 0.18% eROI2 = -0.35% eROI3 = -0.96% mSpread = 0.0114%
Epoch = 7. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.8
RD = 13 NZ = 16 NZA = 19 pNZ = 0.014% pNZA = 0.016% AD = 81.25% ADA = 68.42% NO = 14 GSP = 85.71% N

C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\fromnumeric.py:3194: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch = 7. Time index = 1. Threshold MC = 0.5. Threshold MD = 0.5
J_test = 4.5608315, J_train = 4.2515653775989675, Accuracy=0.680232856775961
RD = 826 NZ = 1516 NZA = 2462 pNZ = 1.298% pNZA = 2.108% AD = 54.49% ADA = 33.55% NO = 321 GSP = 55.14% NSP = 44.86%
SI2 = -44.55 SI = -32.55 eGROI = -0.65% eROI = -4.53% eROI2 = -7.07% eROI3 = -10.28% mSpread = 0.0121%
Epoch = 7. Time index = 1. Threshold MC = 0.5. Threshold MD = 0.6
RD = 267 NZ = 434 NZA = 727 pNZ = 0.372% pNZA = 0.622% AD = 61.52% ADA = 36.73% NO = 183 GSP = 59.56% NSP = 51.37%
SI2 = -11.65 SI = -6.65 eGROI = 3.73% eROI = 1.39% eROI2 = 0.07% eROI3 = -1.76% mSpread = 0.0128%
Epoch = 7. Time index = 1. Threshold MC = 0.5. Threshold MD = 0.7
RD = 113 NZ = 154 NZA = 251 pNZ = 0.132% pNZA = 0.215% AD = 73.38% ADA = 45.02% NO = 88 GSP = 70.45% NSP = 64.77%
SI2 = 6.60 SI = 8.60 eGROI = 2.45% eROI = 1.30% eROI2 = 0.69% eROI3 = -0.19% mSpread = 0.0130%
Epoch = 7. Time index = 1. Threshold MC = 0.5. Threshold MD = 0.8
RD = 23 NZ = 31 N

C:\Users\mgutierrez\root\Projects\SDC\py\git\results2.py:619: RuntimeWarning: invalid value encountered in double_scalars
  eROIs = eROIs+GROI-fixed_spread_ratios


Epoch = 9. Time index = 0. Threshold MC = 0.8. Threshold MD = 0.8
RD = 1 NZ = 1 NZA = 1 pNZ = 0.001% pNZA = 0.001% AD = 100.00% ADA = 100.00% NO = 1 GSP = 100.00% NSP = 100.00%
SI2 = -0.55 SI = 0.45 eGROI = 0.08% eROI = 0.02% eROI2 = 0.06% eROI3 = 0.05% mSpread = 0.0595%
Epoch = 9. Time index = 0. Threshold MC = 0.8. Threshold MD = 0.9
RD = 1 NZ = 1 NZA = 1 pNZ = 0.001% pNZA = 0.001% AD = 100.00% ADA = 100.00% NO = 1 GSP = 100.00% NSP = 100.00%
SI2 = -0.55 SI = 0.45 eGROI = 0.08% eROI = 0.02% eROI2 = 0.06% eROI3 = 0.05% mSpread = 0.0595%

Epoch = 9. Time index = 0. Threshold MC = 0.9. Threshold MD = 0.5
RD = 1 NZ = 1 NZA = 1 pNZ = 0.001% pNZA = 0.001% AD = 100.00% ADA = 100.00% NO = 1 GSP = 100.00% NSP = 100.00%
SI2 = -0.55 SI = 0.45 eGROI = 0.08% eROI = 0.02% eROI2 = 0.06% eROI3 = 0.05% mSpread = 0.0595%
Epoch = 9. Time index = 0. Threshold MC = 0.9. Threshold MD = 0.6
RD = 1 NZ = 1 NZA = 1 pNZ = 0.001% pNZA = 0.001% AD = 100.00% ADA = 100.00% NO = 1 GSP = 100.00% NSP = 100.00%
SI2 = 

KeyboardInterrupt: 

## Evaluate Results

In [68]:
import pandas as pd
from kaissandra.results2 import get_extended_results

list_results_names = ['100318INVO','100318R20','100318R20']
results_dir  = '../../RNN/results/'
ext = '.csv'
list_epochs = [15,29,28]
list_tis = [1,0,0]
list_mcs = [.7,.5,.5]
list_mds = [.7,.75,.75]
Journal = pd.DataFrame()
for l,name in enumerate(list_results_names):
    journal_filename = results_dir+name+'/journal/J_E'+str(list_epochs[l])+'TI'+str(list_tis[l])+'MC'+str(list_mcs[l])+'MD'+str(list_mds[l])+ext
    Journal = Journal.append(pd.read_csv(journal_filename,sep='\t')).sort_values(by=['Asset','DTi']).reset_index().drop(labels='level_0',axis=1)
    print(journal_filename)
#res_ext, log = get_extended_results(Journal,
#                                    5,
#                                    n_days, resultsDir, IDresults,
#                                    epoch, t_index, thr_mc, thr_md,
#                                    get_positions=False,pNZA=1)

../../RNN/results/100318INVO/journal/J_E15TI1MC0.7MD0.7.csv
../../RNN/results/100318R20/journal/J_E29TI0MC0.5MD0.75.csv
../../RNN/results/100318R20/journal/J_E28TI0MC0.5MD0.75.csv


In [69]:
print("shape")
print(Journal.shape[0])
#print(Journal)
n_days = 33*5+1
pos_dirname = '../../RNN/results/MERGED/positions/'
pos_filename = 'P_100318INVO_100318R20.csv'
res_ext, log = get_extended_results(Journal,
                                    5,
                                    n_days,
                                   get_positions=True,pos_dirname=pos_dirname,
                                   pos_filename=pos_filename)
print(res_ext)


shape
9619


100%|████████████████████████████████████████████████████████████████████████████| 9618/9618 [00:02<00:00, 4061.70it/s]


{'NSP': 39.795421334632245, 'SI1': 385.84999999999985, 'eROI4': -13.359543175662777, 'eROI2': 27.700456824336644, 'eROI5': -33.88954317566317, 'SI2': 178.84999999999985, 'NO': 2053, 'mSpread': 0.04760583652411108, 'NSP3': 52.362396492937165, 'sharpe': -8.929004845609478, 'SI4': -282.1500000000001, 'GSP': 80.71115440818315, 'eGROI': 68.76045682433703, 'eROI': -28.974325559663345, 'NOl2': 396.0, 'NSP5': 31.612274719922066, 'eGl1': 49.33477347201489, 'NOl1': 1657.0, 'SI3': -54.15000000000001, 'eRl1': -30.91222244225491, 'SI5': -480.15000000000003, 'eGl2': 19.42568335232196, 'SI': -312.15000000000003, 'NSP.5': 77.69118363370677, 'eRl2': 1.937896882591518, 'NSP4': 41.256697515830496, 'SI.5': 465.84999999999997, 'eROI3': 7.17045682433702, 'eROI.5': 58.495456824337964, 'NSP1': 73.79444715051144, 'eROI1': 48.23045682433843, 'NSP2': 63.711641500243545}


In [41]:
print("shape")
print(Journal.shape[0])
#print(Journal)
n_days = 33*5+1
pos_dirname = '../../RNN/results/MERGED/positions/'
pos_filename = 'P_100318INVO_100318R20_100318R20.csv'
res_ext, log = get_extended_results(Journal,5,n_days,
                                   get_positions=True,pos_dirname=pos_dirname,
                                   pos_filename=pos_filename)
print(res_ext)

shape
9414


100%|████████████████████████████████████████████████████████████████████████████| 9413/9413 [00:01<00:00, 5046.32it/s]


{'NSP': 40.32863849765258, 'SI1': 384.49999999999983, 'eROI4': -14.557696423722136, 'eROI2': 28.04230357627778, 'eROI5': -35.85769642372226, 'SI2': 163.50000000000003, 'NO': 2130, 'mSpread': 0.046841578435802673, 'NSP3': 51.97183098591549, 'sharpe': -8.905498923930248, 'SI4': -303.50000000000017, 'GSP': 80.46948356807512, 'eGROI': 70.64230357627794, 'eROI': -29.130258491982087, 'NOl2': 396.0, 'NSP5': 31.314553990610328, 'eGl1': 51.051955418111156, 'NOl1': 1734.0, 'SI3': -64.50000000000016, 'eRl1': -31.337225493286535, 'SI5': -504.5000000000001, 'eGl2': 19.590348158166634, 'SI': -312.5000000000001, 'NSP.5': 77.1830985915493, 'eRl2': 2.2069670013043896, 'NSP4': 40.751173708920184, 'SI.5': 472.49999999999983, 'eROI3': 6.742303576277957, 'eROI.5': 59.99230357627901, 'NSP1': 73.05164319248826, 'eROI1': 49.34230357627945, 'NSP2': 62.676056338028175}


In [76]:
positions = pd.read_csv(pos_dirname+pos_filename,sep='\t')
#print(positions)
pip_limit = 0.02
pos_under_2p = positions['spread']<pip_limit
per_under_2p = 100*sum(pos_under_2p)/positions.shape[0]
tgsr = 100*sum(positions['GROI']>0)/positions.shape[0]
gsr = 100*sum(positions[pos_under_2p]['GROI']>0)/sum(pos_under_2p)
tsr = 100*sum(positions['ROI']>0)/positions.shape[0]
sr = 100*sum(positions[pos_under_2p]['ROI']>0)/sum(pos_under_2p)
mean_spread = 100*positions[pos_under_2p]['spread'].mean()
print("mean_spread")
print(mean_spread)
print("Number of pos under "+str(pip_limit))
print(positions[pos_under_2p].shape[0])
print("per under pip_limit")
print(per_under_2p)
print("total gross success rate")
print(tgsr)
print("gross success rate")
print(gsr)
print("total success rate")
print(tsr)
print("success rate")
print(sr)
print("ROI for positions under "+str(pip_limit))
print(positions[pos_under_2p]['ROI'].sum())
print("positions['GROI'].sum()-pip_limit*positions['GROI'].shape[0]")
print(positions['GROI'].sum()-pip_limit*positions['GROI'].shape[0])

mean_spread
1.3893650793650802
Number of pos under 0.02
378
per under pip_limit
18.412079883097906
total gross success rate
80.71115440818315
gross success rate
76.19047619047619
total success rate
39.405747686312715
success rate
64.55026455026454
ROI for positions under 0.02
6.5178
positions['GROI'].sum()-pip_limit*positions['GROI'].shape[0]
27.698800000000006


In [40]:
positions = pd.read_csv(pos_dirname+pos_filename,sep='\t')
#print(positions)
pip_limit = 0.03
pos_under_2p = positions['spread']<pip_limit
per_under_2p = 100*sum(pos_under_2p)/positions.shape[0]
tgsr = 100*sum(positions['GROI']>0)/positions.shape[0]
gsr = 100*sum(positions[pos_under_2p]['GROI']>0)/sum(pos_under_2p)
tsr = 100*sum(positions['ROI']>0)/positions.shape[0]
sr = 100*sum(positions[pos_under_2p]['ROI']>0)/sum(pos_under_2p)
mean_spread = 100*positions[pos_under_2p]['spread'].mean()
print("mean_spread")
print(mean_spread)
print("positions[pos_under_2p].shape[0]")
print(positions[pos_under_2p].shape[0])
print("per under pip_limit")
print(per_under_2p)
print("tgsr")
print(tgsr)
print("gsr")
print(gsr)
print("tsr")
print(tsr)
print("sr")
print(sr)
print("ROI for positions under "+str(pip_limit))
print(positions[pos_under_2p]['ROI'].sum())
print("positions['GROI'].sum()-pip_limit*positions['GROI'].shape[0]")
print(positions['GROI'].sum()-pip_limit*positions['GROI'].shape[0])

mean_spread
1.9215471698113193
positions[pos_under_2p].shape[0]
795
per under pip_limit
37.32394366197183
tgsr
80.46948356807512
gsr
76.60377358490567
tsr
39.906103286384976
sr
58.61635220125786
ROI for positions under 0.03
8.3522
positions['GROI'].sum()-pip_limit*positions['GROI'].shape[0]
6.741400000000006
